In [1]:
from gensim.models import KeyedVectors


zh_model = KeyedVectors.load_word2vec_format('zh.vec')
words = []
for word in zh_model.vocab:
    words.append(word)
print("预训练模型包含单词总数： {}".format(len(words)))
words[:10]

D:\Anaconda\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


预训练模型包含单词总数： 50102


['的', '</s>', '在', '是', '年', '和', '了', '於', '為', '有']

In [2]:
print("词向量维度: {}".format(
    len(zh_model[words[0]])
))
len(words)
#print("单词“{}”的向量表示: {}".format(words[0], zh_model[words[0]]))



词向量维度: 300


50102

In [3]:
for similar_word in zh_model.similar_by_word('傻', topn=5):
    print("Word: {0}, Similarity: {1:.2f}".format(
        similar_word[0], similar_word[1]
    ))

Word: 搞笑, Similarity: 0.48
Word: 笑, Similarity: 0.47
Word: 白痴, Similarity: 0.46
Word: 害羞, Similarity: 0.45
Word: 笨蛋, Similarity: 0.45


D:\Anaconda\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [5]:
#  神经网络部分
'''
首先把微博文本打标签  关联微博url 
然后做成dataframe 两列  一列是x 一列是 y


'''

'\n首先把微博文本打标签  关联微博url \n然后做成dataframe 两列  一列是x 一列是 y\n\n\n'

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import re

def read_glove_vecs(glove_file):
    with open(glove_file, 'r') as f:
        words = set()
        word_to_vec_map = {}
        for line in f:
            line = line.strip().split()
            curr_word = line[0]
            words.add(curr_word)
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)

        i = 1
        words_to_index = {}
        index_to_words = {}
        for w in sorted(words):
            words_to_index[w] = i
            index_to_words[i] = w
            i = i + 1
    return words_to_index, index_to_words, word_to_vec_map

def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()


def read_csv(filename = 'data/emojify_data.csv'):
    phrase = []
    emoji = []

    with open (filename) as csvDataFile:
        csvReader = csv.reader(csvDataFile)

        for row in csvReader:
            phrase.append(row[0])
            emoji.append(row[1])

    X = np.asarray(phrase)
    Y = np.asarray(emoji, dtype=int)

    return X, Y
'''
def convert_to_one_hot(Y, C):
    Y = np.eye(C)[Y.reshape(-1)]
    return Y
'''

emoji_dictionary = {"0": "\u2764\uFE0F",    # :heart: prints a black instead of red heart depending on the font
                    "1": ":baseball:",
                    "2": ":smile:",
                    "3": ":disappointed:",
                    "4": ":fork_and_knife:"}

def label_to_emoji(label):
    """
    Converts a label (int or string) into the corresponding emoji code (string) ready to be printed
    """
    return emoji.emojize(emoji_dictionary[str(label)], use_aliases=True)


def print_predictions(X, pred):
    print()
    for i in range(X.shape[0]):
        print(X[i], label_to_emoji(int(pred[i])))


def plot_confusion_matrix(y_actu, y_pred, title='Confusion matrix', cmap=plt.cm.gray_r):

    df_confusion = pd.crosstab(y_actu, y_pred.reshape(y_pred.shape[0],), rownames=['Actual'], colnames=['Predicted'], margins=True)

    df_conf_norm = df_confusion / df_confusion.sum(axis=1)

    plt.matshow(df_confusion, cmap=cmap) # imshow
    #plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(df_confusion.columns))
    plt.xticks(tick_marks, df_confusion.columns, rotation=45)
    plt.yticks(tick_marks, df_confusion.index)
    #plt.tight_layout()
    plt.ylabel(df_confusion.index.name)
    plt.xlabel(df_confusion.columns.name)


def predict(X, Y, W, b, word_to_vec_map):
    """
    Given X (sentences) and Y (emoji indices), predict emojis and compute the accuracy of your model over the given set.

    Arguments:
    X -- input data containing sentences, numpy array of shape (m, None)
    Y -- labels, containing index of the label emoji, numpy array of shape (m, 1)

    Returns:
    pred -- numpy array of shape (m, 1) with your predictions
    """
    m = X.shape[0]
    pred = np.zeros((m, 1))

    for j in range(m):                       # Loop over training examples

        # Split jth test example (sentence) into list of lower case words
        words = X[j].lower().split()

        # Average words' vectors
        avg = np.zeros((50,))
        for w in words:
            avg += word_to_vec_map[w]
        avg = avg/len(words)

        # Forward propagation
        Z = np.dot(W, avg) + b
        A = softmax(Z)
        pred[j] = np.argmax(A)

    print("Accuracy: "  + str(np.mean((pred[:] == Y.reshape(Y.shape[0],1)[:]))))

    return pred


In [6]:
#准备数据集

df = pd.read_excel('D:/语料1.xls')
df = df[['text','tag']]

    

In [7]:
df2 = pd.read_excel("D:/税微博7月1-10改.xlsx")
df2 = df2[['text']]

In [114]:
#打分小程序
df2 = df2[:10]
df2['fenshu']= None
def dafen(df):
    for i in range(1,len(df)):
        print(df.loc[i][0])
        fenshu = input('打分：')
        df.loc[i]['fenshu'] = fenshu
        
dafen(df2[:10
        ])

#世界杯赌球#来来来交交智商税07月01日00:59来自三星GALAX
打分：
杜兰特可能选择1+1，明年夏天有选择权跳出，并且有鸟权可以签5年顶薪。简单说就是杜兰特损失18-19的500万，从3500万变成3000万，因为签1+1只有前一年120%的涨幅，换回明年5年顶薪的机会。顺便下赛季勇士省点税，和多签人没关系07月01日00:57来自Rozier’siPhone客
打分：
中国房价最高城市开征空置税！雷霆手段控楼市即将到来？|地产·杆(分享自@Flipboard红板报)O网页链接07月01日00:57来自iPhone客
打分：
宁波市“月荷税莲”名家书法展于月至日在鼓楼展出，本次展出由海曙国税局主办，欢迎参观。07月01日00:57来自微博weibo.c
打分：
【2014年共3.94万辆新能源汽车免征购置税】据国家税务总局统计，2014年9月-12月有29个省市有办理新能源汽车免征车购税手续，共免征车购税新能源汽车3.94万辆，其中乘用车3.38万辆，商用车5615辆。07月01日00:54来自小米Note3拍人
打分：
天惹，可爱！//@H-HeeBum王大拿:我被可爱死了//@Auroraborealis-:啊太可爱了//@SnowRiver秘密日報社:必須上可愛稅！//@Alpha帅锤:可爱//@Loki夫人:请上可爱税谢谢//@VelvetBobby:软fufu//@Wally不吃咸鸭蛋:太可爱了07月01日00:54来自iPhone8Pl
打分：
⭐应届生求职招聘简历五险一金养老保险医疗保险生育保险工伤保险失业保险公积金贷款买房面试技巧签合同三方协议社保报销工资薪酬年终奖月薪年薪绩效业绩指标谈判劳动仲裁毕业生档案报到证登记证单身税失业证朱正廷@THEO-朱正廷07月01日00:53来自倚栈的Andro
打分：
//@Vera南夏:呜哇//@卉子_ForPaul:呜呜呜呜可爱哭了！！！//@MissyWow:我的妈我要这个男人！//@汤抖森的猫耳:啊啊啊嘤嘤嘤嘤嘤嘤嘤嘤嘤(ಥ_ಥ)//@Alpha帅锤:可爱//@Loki夫人:请上可爱税谢谢//@VelvetBobby:软fufu//@Wally不吃咸鸭蛋:太可爱了07月01日00:52来自iPhone客
打分：
可爱(๑•.•๑)//@Alpha帅锤:可爱//@Loki夫人:请上可爱税谢谢//@V

In [155]:
df2 = df2[:8]
df2

,text,fenshu
1,#世界杯赌球#来来来交交智商税07月01日00:59来自三星GALAX,None
2,杜兰特可能选择1+1，明年夏天有选择权跳出，并且有鸟权可以签5年顶薪。简单说就是杜兰特损失1...,None
3,中国房价最高城市开征空置税！雷霆手段控楼市即将到来？|地产·杆(分享自@Flipboard红...,None
4,宁波市“月荷税莲”名家书法展于月至日在鼓楼展出，本次展出由海曙国税局主办，欢迎参观。07月0...,None
5,【2014年共3.94万辆新能源汽车免征购置税】据国家税务总局统计，2014年9月-12月有...,None
6,天惹，可爱！//@H-HeeBum王大拿:我被可爱死了//@Auroraborealis-:...,None
7,⭐应届生求职招聘简历五险一金养老保险医疗保险生育保险工伤保险失业保险公积金贷款买房面试技巧签...,None
8,//@Vera南夏:呜哇//@卉子_ForPaul:呜呜呜呜可爱哭了！！！//@MissyW...,None


In [8]:


X_train = np.array(df['text'])[20:]
Y_train = np.array(df['tag'])[20:]
X_test = np.array(df['text'])[:20]
Y_test = np.array(df['tag'])[:20]

print(X_train,Y_train,X_test,Y_test)


['感觉到幸福' '无限扩大' '坚定不移' '中二垃圾少年' '分享视频' '简直泪目' '情绪失控' '为他跳起毛利战舞' '最后一个真喜庆'
 '又帅又有才' '斗气' '晒太阳这么舒服' '这位女士去了动物园' '陪伴和温暖' '接触陶瓷已经有二十年' '最强美男子' '将该男生抓获'
 '发生了不好的事' '感到同情' '该男生心理有问题' '去年买的衣服' '纯真自由的生活' '不让自己过于后悔' '家暴不是犯错' '更加珍惜'
 '十分恐惧' '伤心难过的时候' '安慰他们' '给予他们支持' '一个好消息' '去芬兰的视频' '上学' '经典的存在' '增加幸福感'
 '羊毛裙' '海马毛珊瑚绒家居服' '收集温暖、治愈、幸运消息的版块' '没抛弃你' '组织学生撤离' '慌乱之中' '相信友情' '他是个好人'
 '斗罗大陆' '转发抽奖' '他真的很努力' '领取本月星座运势秘籍' '父母老去' '信任和照顾' '谣言' '不要错过'
 '考名牌大学是所有家长的希' '活泼开朗' '能把科技唠成家常' '花很多时间' '经历人间的生离死别' '明年出场的新本子' '继续保持可爱'
 '出现过很多次' '最喜爱的作家' '消防员将呼吸器让给婴儿' '同一个世界' '吃瓜群众' '当众掌掴辱骂3名学生' '闻风而逃'
 '更多孩子在欧洲各地营地被陆续发现' '人类之耻' '最完美的血统' '压抑的怒气' '在众人的簇拥下游街示众' '令挪威人害怕'
 '只能忍受更加无常的命运' '最幸福的日子' '仇恨' '战争仍在继续' '上面刻着他的名字' '大屠杀资料馆' '对拍摄的居民身份证相片不满意'
 '心地善良' '突然刷爆朋友圈' '梦见好多好多三明治'] [ 1  0  1 -2  0 -1 -2  0  1  1 -1  1  0  2  0  1 -1 -1 -1 -2  0  1  1 -2
  1 -1 -1  1  2  2  0  0  1  2  0  0  2 -1  0 -1  1  2  0  0  1  0 -1  2
 -1 -1  1  2  0  0 -2  0  1  0  1  0  0 -1 -2 -1  0 -2  2 -2 -2 -1 -1  2
 -2 -1  0 -2 -1  1  0  0] ['今晚起1

In [9]:
Y = '2'
Y = np.eye(5)[int(Y)+2]
Y

array([0., 0., 0., 0., 1.])

In [10]:
maxLen = len(max(X_train, key=len).split())


In [11]:
word = '搞笑'
zh_model[word]

array([ 0.25859  ,  0.22439  , -0.050759 , -0.043929 , -0.12326  ,
       -0.13298  ,  0.38223  ,  0.028844 , -0.45969  , -0.11023  ,
       -0.28477  ,  0.033635 ,  0.24709  ,  0.096075 ,  0.19565  ,
        0.25969  , -0.14721  , -0.38603  ,  0.33876  ,  0.32904  ,
       -0.1708   , -0.15786  , -0.025603 ,  0.059944 , -0.33972  ,
       -0.099513 ,  0.080611 ,  0.73935  ,  0.20882  ,  0.22658  ,
        0.40455  , -0.022722 ,  0.33032  , -0.24389  ,  0.26036  ,
        0.30444  , -0.42477  ,  0.23229  ,  0.28836  ,  0.34982  ,
        0.19008  ,  0.15437  , -0.17395  , -0.035459 , -0.083979 ,
       -0.10536  ,  0.49609  ,  0.08135  , -0.07996  , -0.38033  ,
       -0.18951  , -0.11024  , -0.018996 , -0.24443  , -0.13062  ,
       -0.19577  , -0.1743   ,  0.25094  ,  0.12162  ,  0.18115  ,
       -0.23824  ,  0.26407  , -0.38566  , -0.1113   ,  0.25321  ,
        0.10368  ,  0.099715 , -0.060418 , -0.10433  , -0.14365  ,
       -0.0098679, -0.52156  ,  0.081901 ,  0.037713 ,  0.0318

In [12]:
import jieba

#把句子的向量均值化

sen = '我超级喜欢纯子啊'
def sen_to_avg(sen):
    sen =re.sub(r"[\s+\.\!\/_,$%^*(【】：\]\[\-:;+\"\']+|[+——！，。？、~@#￥%……&*（）]+|[0-9]+","",sen)
            
    l = jieba.lcut(sen)
    #print(l)
    avg = np.zeros((300,))
    n = 0
    for w in l:
        if w in words:
            avg += zh_model[w]
        else:
            avg = avg
            n = n+1
    if len(l)-n == 0:
        avg = avg
    else:
        avg = avg/(len(l)-n)
    #print(len(l)-n)
    return avg
        
    
#把标签转化为 one—hot 
def convert_to_one_hot(Y, C):
    Y = np.eye(C)[int(Y)+2]
    return Y




In [13]:
def predict(X, Y, W, b):
    """
    Given X (sentences) and Y (emoji indices), predict emojis and compute the accuracy of your model over the given set.

    Arguments:
    X -- input data containing sentences, numpy array of shape (m, None)
    Y -- labels, containing index of the label emoji, numpy array of shape (m, 1)

    Returns:
    pred -- numpy array of shape (m, 1) with your predictions
    """
    m = X.shape[0]
    pred = np.zeros((m, 1))

    for j in range(m):                       # Loop over training examples

        # Split jth test example (sentence) into list of lower case words
        avg = sen_to_avg(X[j])

        # Forward propagation
        Z = np.dot(W, avg) + b
        A = softmax(Z)
        pred[j] = np.argmax(A)-2
        print(pred[j])
        

    print("Accuracy: "  + str(np.mean((pred[:] == Y.reshape(Y.shape[0],1)[:]))))

    return pred


In [14]:
# 模型部分

def model(X,Y,learning_rate = 0.01,num_iterations = 400):
    np.random.seed(1)
    
    m = Y.shape[0]  #训练集数量
    n_y = 5   #标签的个数  softmax的类别数
    n_h = 300  #词向量的维度数
    
    W = np.random.randn(n_y,n_h)/np.sqrt(n_h)
    
    b = np.zeros((n_y,))
    
    
    for t in range(num_iterations):
        for i in range(m):
            
            
            avg = sen_to_avg(X[i])
            #print(i,avg)
            #前向
            z = np.dot(W,avg)+b
            a = softmax(z)
            
            
            #计算损失
            Y_oh = convert_to_one_hot(Y[i],n_y)
            #print(Y_oh)
            cost = -np.sum(Y_oh * np.log(a))
            
            
            #下降参数
            dz = a - Y_oh
            dw = np.dot(dz.reshape(n_y,1), avg.reshape(1, n_h))
            db = dz
            
            #反向传播
            W = W - learning_rate*dw
            b = b - learning_rate*db
            
        if t%100 ==0:
            print("Epoch: " + str(t) + " --- cost = " + str(cost))
            pred = predict(X, Y, W, b)
            
    return W,b
W,b = model(X_train,Y_train,num_iterations = 400)

Building prefix dict from the default dictionary ...
Loading model from cache D:\Temp\jieba.cache
Loading model cost 0.935 seconds.
Prefix dict has been built succesfully.


Epoch: 0 --- cost = 1.6302944374124193
[0.]
[0.]
[0.]
[-1.]
[0.]
[-1.]
[0.]
[0.]
[0.]
[1.]
[0.]
[-1.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[1.]
[0.]
[-1.]
[0.]
[-1.]
[-1.]
[1.]
[-1.]
[0.]
[-1.]
[0.]
[0.]
[0.]
[0.]
[0.]
[1.]
[0.]
[0.]
[0.]
[0.]
[0.]
[-1.]
[-2.]
[-1.]
[0.]
[0.]
[-1.]
[-2.]
[-1.]
[-1.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[1.]
[0.]
[0.]
[1.]
[0.]
[0.]
[0.]
[0.]
[1.]
[1.]
[-1.]
Accuracy: 0.3375
Epoch: 100 --- cost = 0.036406346748235285
[1.]
[0.]
[-1.]
[-2.]
[0.]
[-1.]
[-2.]
[0.]
[1.]
[1.]
[-1.]
[1.]
[0.]
[2.]
[0.]
[1.]
[0.]
[-1.]
[-1.]
[-2.]
[0.]
[1.]
[-1.]
[-2.]
[1.]
[-1.]
[-1.]
[1.]
[2.]
[2.]
[0.]
[0.]
[1.]
[2.]
[0.]
[0.]
[2.]
[-1.]
[0.]
[-1.]
[1.]
[2.]
[0.]
[0.]
[1.]
[0.]
[-1.]
[2.]
[-1.]
[-1.]
[1.]
[-1.]
[0.]
[0.]
[-2.]
[0.]
[1.]
[0.]
[1.]
[0.]
[0.]
[-1.]
[-2.]
[-1.]
[0.]
[-2.]
[2.]
[-2.]
[-1.]
[-1.]
[-1.]
[2.]
[-2.]
[-1.]
[0.]
[-2.]
[-1.]
[1.]
[0.]
[0.]
Accuracy: 0.9375
Epoch: 200 --- cost = 0.0117175788781

In [20]:


predict(X_test,Y_test,W,b)

[0.]
[0.]
[0.]
[-1.]
[1.]
[2.]
[0.]
[-2.]
[1.]
[1.]
[-1.]
[1.]
[1.]
[0.]
[-2.]
[1.]
[0.]
[-1.]
[0.]
[0.]
Accuracy: 0.45


array([[ 0.],
       [ 0.],
       [ 0.],
       [-1.],
       [ 1.],
       [ 2.],
       [ 0.],
       [-2.],
       [ 1.],
       [ 1.],
       [-1.],
       [ 1.],
       [ 1.],
       [ 0.],
       [-2.],
       [ 1.],
       [ 0.],
       [-1.],
       [ 0.],
       [ 0.]])